In [1]:
import os
os.chdir('..') # this resolves ImportError: attempted relative import with no known parent package

# general DS packages
import pandas as pd
import numpy as np

# cleaning and pre-processing
from src.processing.text_cleaning import normalize_text, process_contractions, remove_all_punctuation, remove_emojis, remove_html_unescape, remove_digits, remove_extra_whitespace, remove_website_links
from src.processing.text_processing import tokenize_comment, lemmatize_comment, remove_stop_words, part_of_speech, part_of_speech_tag, part_of_speech_dependency, part_of_speech_shape, part_of_speech_alpha, part_of_speech_is_stop

# modeling
import gensim
from gensim import corpora
from gensim.models import LsiModel
from gensim.models import LdaModel

# visualisation
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import data from csv
raw_data = pd.read_csv("data/raw/new_character_reveal_comments.csv", )
data = raw_data.copy()
df = pd.DataFrame(data)

# Clean and process dataframe

In [3]:
%%time

# clean
df['textDisplay'] = df['textDisplay'].apply(normalize_text)
df['textDisplay'] = df['textDisplay'].apply(process_contractions)
df['textDisplay'] = df['textDisplay'].apply(remove_website_links)
df['textDisplay'] = df['textDisplay'].apply(remove_html_unescape)
df['textDisplay'] = df['textDisplay'].apply(remove_emojis)
df['textDisplay'] = df['textDisplay'].apply(remove_digits)
df['textDisplay'] = df['textDisplay'].apply(remove_all_punctuation)
df['textDisplay'] = df['textDisplay'].apply(remove_extra_whitespace)

# process
df["textStopWordsRemoved"] = df["textDisplay"].apply(remove_stop_words)
df["textTokenized"] = df['textStopWordsRemoved'].apply(tokenize_comment)
df["textLemmatized"] = df["textStopWordsRemoved"].apply(lemmatize_comment)

# part of speech operations
(df["pos"], 
 df["posTag"],
 df["posDependency"],
 df["posShape"],
 df["posAlpha"],
 df["posStopWord"]) = (df["textStopWordsRemoved"].apply(part_of_speech),
                                 df["textStopWordsRemoved"].apply(part_of_speech_tag),
                                 df["textStopWordsRemoved"].apply(part_of_speech_dependency),
                                 df["textStopWordsRemoved"].apply(part_of_speech_shape),
                                 df["textStopWordsRemoved"].apply(part_of_speech_alpha),
                                 df["textStopWordsRemoved"].apply(part_of_speech_is_stop)
                                )
df.head()

CPU times: user 38.2 s, sys: 33.5 ms, total: 38.2 s
Wall time: 38.3 s


,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay,textStopWordsRemoved,textTokenized,textLemmatized,pos,posTag,posDependency,posShape,posAlpha,posStopWord
0,rDxrpSqYHD8,@silveriver9,2023-11-01 16:09:58+00:00,2023-11-01 16:10:43+00:00,4,4,first now where is lei wulong,lei wulong,"[lei, wulong]","[lei, wulong]","[PROPN, NOUN]","[NNP, NN]","[compound, ROOT]","[xxx, xxxx]","[True, True]","[False, False]"
1,rDxrpSqYHD8,@faizaanjaved7150,2023-11-01 16:10:05+00:00,2023-11-01 16:10:05+00:00,1,1,already seen it you are getting less views now...,seen getting views bamco,"[seen, getting, views, bamco]","[see, get, view, bamco]","[VERB, VERB, NOUN, NOUN]","[VBN, VBG, NNS, NNS]","[ROOT, xcomp, dobj, dobj]","[xxxx, xxxx, xxxx, xxxx]","[True, True, True, True]","[False, False, False, False]"
2,rDxrpSqYHD8,@TS-rw4lk,2023-11-01 16:10:05+00:00,2023-11-01 16:10:05+00:00,0,0,wow,wow,[wow],[wow],[INTJ],[UH],[ROOT],[xxx],[True],[False]
3,rDxrpSqYHD8,@ALONCAK,2023-11-01 16:10:06+00:00,2023-11-01 16:10:06+00:00,0,0,oww yeaah,oww yeaah,"[oww, yeaah]","[oww, yeaah]","[PROPN, PROPN]","[NNP, NNP]","[compound, ROOT]","[xxx, xxxx]","[True, True]","[False, False]"
4,rDxrpSqYHD8,@Rough_Estimates,2023-11-01 16:10:06+00:00,2023-11-01 16:10:06+00:00,135,14,i hope we get an angel version of jin,hope angel version jin,"[hope, angel, version, jin]","[hope, angel, version, jin]","[PROPN, PROPN, PROPN, PROPN]","[NNP, NNP, NNP, NNP]","[compound, compound, compound, ROOT]","[xxxx, xxxx, xxxx, xxx]","[True, True, True, True]","[False, False, False, False]"


# Topic modeling

1. Potentially create bigram and trigram models.


## Create corpus and document-term matrix

Create a corpus (a list that contains all of the YouTube comments in it))...

In [15]:
# create a corpus
corpus = list(df['textLemmatized'].values)
corpus

[['lei', 'wulong'],
 ['see', 'get', 'view', 'bamco'],
 ['wow'],
 ['oww', 'yeaah'],
 ['hope', 'angel', 'version', 'jin'],
 ['begin'],
 ['let'],
 ['miguel', 'waiting', 'room', 'right'],
 ['wow'],
 ['yo'],
 ['marvelous'],
 ['late', 'bandai'],
 ['excitement'],
 ['marvelous'],
 ['new', 'trailer', 'drop', 'excited', 'game'],
 ['need', 'rest', 'trailer', 'mfs'],
 ['background', 'music', 'trailer', 'bchef', 'kissb'],
 ['reason', 'm', 'get', 'ps'],
 ['primer', 'comentario'],
 ['legend', 'shiiet', 'throw', 'lei', 'st'],
 ['character', 'bandai', 'go', 'love'],
 ['good', 'fight', 'game', 'franchise'],
 ['want', 'steve', 'action', 'point'],
 ['yoshimitsu', 'get', 'trailer', 'crime'],
 ['know', 'devil', 'jin', 'shriek', 'like', 'thatt'],
 ['hrefvrdxrpsqyhdtmsa'],
 ['amazing'],
 ['hailo', 'bandai', 'sayonara'],
 ['finally', 'lee', 'sadly', 'design', 'suck'],
 ['think', 'new', 'trailer'],
 ['devil', 'jin', 'vs', 'devil', 'kazuya', 'go', 'surreal'],
 ['lucky', 'chloe'],
 ['hope', 'bring', 'character', 

Use Gensim to create a dictionary that will store each word in the corpus and assign a unique ID to it. Then create a bag of words document-term matrix which will return a tuple with the word's unique ID and how many times it occurs in the document i.e., (word_id, frequnecy_in_document)

In [26]:
# create dictionary
comments_dictionary = corpora.Dictionary(corpus)

# create term document frequency
document_term_matrix = [comments_dictionary.doc2bow(doc) for doc in corpus]

In [27]:
comments_dictionary

In [28]:
document_term_matrix

[[(0, 1), (1, 1)],
 [(2, 1), (3, 1), (4, 1), (5, 1)],
 [(6, 1)],
 [(7, 1), (8, 1)],
 [(9, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1)],
 [(14, 1)],
 [(15, 1), (16, 1), (17, 1), (18, 1)],
 [(6, 1)],
 [(19, 1)],
 [(20, 1)],
 [(21, 1), (22, 1)],
 [(23, 1)],
 [(20, 1)],
 [(24, 1), (25, 1), (26, 1), (27, 1), (28, 1)],
 [(28, 1), (29, 1), (30, 1), (31, 1)],
 [(28, 1), (32, 1), (33, 1), (34, 1), (35, 1)],
 [(3, 1), (36, 1), (37, 1), (38, 1)],
 [(39, 1), (40, 1)],
 [(0, 1), (41, 1), (42, 1), (43, 1), (44, 1)],
 [(21, 1), (45, 1), (46, 1), (47, 1)],
 [(26, 1), (48, 1), (49, 1), (50, 1)],
 [(51, 1), (52, 1), (53, 1), (54, 1)],
 [(3, 1), (28, 1), (55, 1), (56, 1)],
 [(11, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1)],
 [(62, 1)],
 [(63, 1)],
 [(21, 1), (64, 1), (65, 1)],
 [(66, 1), (67, 1), (68, 1), (69, 1), (70, 1)],
 [(27, 1), (28, 1), (71, 1)],
 [(11, 1), (46, 1), (57, 2), (72, 1), (73, 1), (74, 1)],
 [(75, 1), (76, 1)],
 [(10, 1), (45, 1), (77, 1), (78, 1)],
 [(79, 1)],
 [],
 [(46, 1), (80, 

The document term matrix is a list and each document (YouTube comment) within the list is now a list of tuples instead of tokens (words). The tuples provide the word ID (each word has a unique ID created by Gensim) and the frequency with which that word occurs in the document.


We can also view the actual word with the frequency...

In [29]:
comments_dictionary[0]

'lei'

In [31]:
# human readable format of document term matrix for first youtube comments
[[(comments_dictionary[id], freq) for id, freq in cp] for cp in document_term_matrix[:10]]

[[('lei', 1), ('wulong', 1)],
 [('bamco', 1), ('get', 1), ('see', 1), ('view', 1)],
 [('wow', 1)],
 [('oww', 1), ('yeaah', 1)],
 [('angel', 1), ('hope', 1), ('jin', 1), ('version', 1)],
 [('begin', 1)],
 [('let', 1)],
 [('miguel', 1), ('right', 1), ('room', 1), ('waiting', 1)],
 [('wow', 1)],
 [('yo', 1)]]

We now have the data in a state where we can build the topic model...

# Build the models

In [61]:
gensim.__version__

'4.3.2'

## LSA

In [75]:
# LSA model
lsa_number_of_topics = 3
lsa_model = LsiModel(corpus=document_term_matrix, 
                     num_topics=lsa_number_of_topics, 
                     id2word=comments_dictionary,
                    random_seed=42)

# LSA model
print(lsa_model.print_topics(num_topics=5, num_words=5))

[(0, '0.501*"tekken" + 0.500*"lei" + 0.500*"wulong" + 0.499*"want" + 0.001*"character"'), (1, '1.000*"capoeira" + 0.000*"game" + 0.000*"fight" + 0.000*"eddy" + 0.000*"s"'), (2, '0.708*"ogre" + 0.706*"jinpachi" + 0.001*"heihachi" + 0.001*"ancient" + 0.001*"like"')]


/Users/jonathonreid/virtual_environments/youtube/env/lib/python3.12/site-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/Users/jonathonreid/virtual_environments/youtube/env/lib/python3.12/site-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/Users/jonathonreid/virtual_environments/youtube/env/lib/python3.12/site-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/Users/jonathonreid/virtual_environments/youtube/env/l

## Perplexity and coherence
- Model perplexity and topic coherence provide a convenient measure to indicate how good a given topic model is.
- Lower the perplexity better the model.
- Higher the topic coherence, the topic is more human interpretable.

In [ ]:
# Compute Perplexity
print(f"\nPerplexity: ' {lsa_model.log_perplexity(chunk=corpus, total_docs=None)}")   # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=comments_dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f"\nCoherence Score: ' {coherence_lda}")

## LDA: bigram and trigram

In [140]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(corpus, min_count=3, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[corpus], threshold=10)  

# Phraser objects take Phrases models as inputs and are optimized for faster phrase application
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[corpus[0]]])   # applies the bigram model to the first document, combining common word pairs. trigram_mod[] applies the trigram model 
                                            # to the result of the bigram application, forming trigrams from frequent bigrams and single words.
print(trigram_mod[bigram_mod[corpus[135]]])

['lei_wulong']
['eddy_gordo']


In [141]:
def make_bigrams(corpus):
    return [bigram_mod[doc] for doc in corpus]

def make_trigrams(corpus):
    return [trigram_mod[bigram_mod[doc]] for doc in corpus]

In [142]:
# create bigrams
corpus_bigrams = make_bigrams(corpus)

In [143]:
# create dictionary
bigrams_dictionary = corpora.Dictionary(corpus_bigrams)

# create term document frequency
bigrams_document_term_matrix = [bigrams_dictionary.doc2bow(doc) for doc in corpus_bigrams]

In [144]:
# LDA model
lda_number_of_topics = 3
lda_bigrams_model = LdaModel(corpus=bigrams_document_term_matrix,
                     num_topics=lda_number_of_topics, 
                     id2word=bigrams_dictionary,
                     passes=10,   # number of iterations over the entire corpus during model training
                    random_state=42,)

# Results
lda.print_topics(num_topics=lda_number_of_topics, num_words=5)

[(2,
  '0.042*"jin" + 0.032*"zafina" + 0.027*"devil" + 0.023*"play" + 0.019*"look"'),
 (3,
  '0.031*"eddy" + 0.030*"jin" + 0.029*"devil" + 0.027*"kuma" + 0.024*"come"'),
 (4,
  '0.064*"tekken" + 0.053*"game" + 0.039*"bring" + 0.025*"eddy" + 0.022*"anna"')]

In [145]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_bigrams_model, bigrams_document_term_matrix, bigrams_dictionary)
vis

/Users/jonathonreid/virtual_environments/youtube/env/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/jonathonreid/virtual_environments/youtube/env/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/jonathonreid/virtual_environments/youtube/env/lib/python3.12/site-packages/joblib/_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
/Users/jonathonreid/virtual_environments/youtube/env/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/jonathonreid/virtual_env

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.007077 -0.192313       1        1  47.036552
2     -0.184441  0.101587       2        1  31.190065
0      0.191518  0.090726       3        1  21.773383, topic_info=          Term        Freq       Total Category  logprob  loglift
288       eddy  165.000000  165.000000  Default  30.0000  30.0000
25        game  146.000000  146.000000  Default  29.0000  29.0000
77       bring  104.000000  104.000000  Default  28.0000  28.0000
45   character   78.000000   78.000000  Default  27.0000  27.0000
27     trailer  138.000000  138.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
349       play   21.752852   59.075046   Topic3  -4.6385   0.5254
57   devil_jin   22.716517  161.998964   Topic3  -4.5951  -0.4400
288       eddy   20.439501  165.365909   Topic3  -4.7008  -0.5662
46          go   19.839438  115.180120   Topic3  -4.7306  -0.2343
681      great   19.274056   51.394556   Topic3  -4.7595   0.5437

[161 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
354       2  0.974364         actual
262       1  0.034407            add
262       3  0.963394            add
90        1  0.986383          alisa
238       2  0.988863           anna
...     ...       ...            ...
204       1  0.996051            wtf
916       3  0.949808         wulong
1341      1  0.992747  wulong_tekken
294       3  0.973658           yeah
84        1  0.992981         zafina

[155 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lsa_model, document_term_matrix, comments_dictionary)
vis